In [2]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.utils import shuffle
from sklearn.preprocessing import FunctionTransformer

# https://erdogant.github.io/distfit/pages/html/Parametric.html

In [3]:
adult = fetch_openml('adult')

c:\Users\vitos\anaconda3\envs\python38\lib\site-packages\sklearn\datasets\_openml.py:292: UserWarning: Multiple active versions of the dataset matching the name adult exist. Versions may be fundamentally different, returning version 1.
  warn(


In [4]:
X = adult['data']
y = adult['target']

In [46]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             48842 non-null  category
 1   workclass       46043 non-null  category
 2   fnlwgt          48842 non-null  float64 
 3   education       48842 non-null  category
 4   education-num   48842 non-null  float64 
 5   marital-status  48842 non-null  category
 6   occupation      46033 non-null  category
 7   relationship    48842 non-null  category
 8   race            48842 non-null  category
 9   sex             48842 non-null  category
 10  capitalgain     48842 non-null  category
 11  capitalloss     48842 non-null  category
 12  hoursperweek    48842 non-null  category
 13  native-country  47985 non-null  category
dtypes: category(12), float64(2)
memory usage: 1.3 MB


In [51]:

num_features = X.select_dtypes(include='number').columns.to_list()
cat_features = X.select_dtypes(exclude='number').columns.to_list()

cardinalitites = [
    X[f].nunique()
    for f in cat_features
]

max_one_hot = 10
oh_cat_features = [cat_features[i] for i,c in enumerate(cardinalitites) if c<=max_one_hot]
ord_cat_features = [cat_features[i] for i,c in enumerate(cardinalitites) if c>max_one_hot]

# Optionally
shuffled_features = X.columns.to_list()[:2]

shuffled_num_features = X[shuffled_features].select_dtypes(include='number').columns.to_list()
shuffled_cat_features = X[shuffled_features].select_dtypes(exclude='number').columns.to_list()

shuffled_cardinalitites = [
    X[f].nunique()
    for f in cat_features
]
shuffled_oh_cat_features = [cat_features[i] for i,c in enumerate(cardinalitites) if c<=max_one_hot]
shuffled_ord_cat_features = [cat_features[i] for i,c in enumerate(cardinalitites) if c>max_one_hot]


['age', 'workclass']

In [52]:
# Generate random features

# plt.hist(
    # np.random.normal(size=(X.shape[0],1))
    # np.random.random(size=(X.shape[0],1))
    # np.random.exponential(size=(X.shape[0],1))
    # np.random.choice([i for i in range(10)], size=(X.shape[0],1))
    #  np.random.binomial(n=1, p=0.7, size=(X.shape[0],1))   
# )

x_norm = pd.DataFrame(np.random.normal(size=(X.shape[0],1)), columns=['x_norm']) 
x_rand = pd.DataFrame(np.random.random(size=(X.shape[0],1)), columns=['x_rand'])
x_exp = pd.DataFrame(np.random.exponential(size=(X.shape[0],1)), columns=['x_exp'])
x_choice = pd.DataFrame(np.random.choice([i for i in range(10)], size=(X.shape[0],1)), columns=['x_choice'])
x_binom = pd.DataFrame(np.random.binomial(n=1, p=0.7, size=(X.shape[0],1))  , columns=['x_binom'])

X_random = pd.concat([x_norm,x_rand,x_exp,x_choice,x_binom], axis=1)

random_features = X_random.columns.to_list()

random_features

['x_norm', 'x_rand', 'x_exp', 'x_choice', 'x_binom']

In [48]:
X_full = pd.concat([
    X,
    X_random
],axis=1)


In [78]:
col_transformer = ColumnTransformer([
    ('oh_cat_features',OneHotEncoder(sparse=False), oh_cat_features),
    ('ord_cat_features',OrdinalEncoder(), ord_cat_features),
    ('num_features', 'passthrough', num_features),
])

shuffle_col_transformer = ColumnTransformer([
    ('shuffled_features', FunctionTransformer(func=shuffle, feature_names_out ='one-to-one'), shuffled_features),
])

random_col_transformer = ColumnTransformer([
    ('random_features', 'passthrough', random_features)  
])


fu = FeatureUnion([
    (
        'fu_col', 
        col_transformer
    ),
    (
        'fu_shuffled_col',
        shuffle_col_transformer
    ),
    (
        'fu_random_col',
        random_col_transformer
    ),
])

# col_transformer.fit_transform(X_full)

X_full_enc = fu.fit_transform(X_full)

feature_names = fu.get_feature_names_out()

In [81]:
X_full_enc
pd.DataFrame(X_full_enc, columns=feature_names)

,fu_col__oh_cat_features__age_0,fu_col__oh_cat_features__age_1,fu_col__oh_cat_features__age_2,fu_col__oh_cat_features__age_3,fu_col__oh_cat_features__age_4,fu_col__oh_cat_features__workclass_Federal-gov,fu_col__oh_cat_features__workclass_Local-gov,fu_col__oh_cat_features__workclass_Never-worked,fu_col__oh_cat_features__workclass_Private,fu_col__oh_cat_features__workclass_Self-emp-inc,...,fu_col__ord_cat_features__native-country,fu_col__num_features__fnlwgt,fu_col__num_features__education-num,fu_shuffled_col__shuffled_features__age,fu_shuffled_col__shuffled_features__workclass,fu_random_col__random_features__x_norm,fu_random_col__random_features__x_rand,fu_random_col__random_features__x_exp,fu_random_col__random_features__x_choice,fu_random_col__random_features__x_binom
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.0,77516.0,13.0,3,Private,-0.666599,0.879247,0.156216,2.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.0,83311.0,13.0,2,Private,-0.032676,0.383668,0.756369,5.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,38.0,215646.0,9.0,1,Private,-0.573323,0.187991,0.574413,3.0,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,38.0,234721.0,7.0,1,Private,0.648376,0.65522,0.006256,6.0,1.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,4.0,338409.0,13.0,1,State-gov,-0.340866,0.36873,0.772984,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,38.0,215419.0,13.0,0,Private,-0.556111,0.939941,4.561541,8.0,0.0
48838,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,38.0,321403.0,9.0,1,Private,-1.258872,0.808261,1.370731,1.0,1.0
48839,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,38.0,374983.0,13.0,3,Private,3.611009,0.512683,0.063033,4.0,0.0
48840,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,38.0,83891.0,13.0,1,Private,0.288912,0.620328,1.352779,3.0,1.0


In [73]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

y_enc = label_encoder.fit_transform(y)


In [80]:
rf = RandomForestClassifier()

rf.fit(X_full_enc, y_enc)

ValueError: could not convert string to float: 'Private'